# Assigment Number 3 - Advanced Programming

### Team Members: Javier Esteban Aragoneses - Mauricio Marcos Fajgenbaun

First, let´s call all the libraries we will need for this assigment.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, PredefinedSplit
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.utils.validation import check_is_fitted

Then, let´s download the dataset given by the professor.

In [4]:
datos=pd.read_pickle('desktop/wind_pickle.pickle')


### Part 1: data setup

First, we will remove the columns: "steps","months","day","hour" from our dataset. And then we make sure, that we haev done so.

In [5]:
datos.drop(['steps','month','day','hour'],axis='columns', inplace=True)
datos.columns

Index(['energy', 'year', 'p54.162.1', 'p54.162.2', 'p54.162.3', 'p54.162.4',
       'p54.162.5', 'p54.162.6', 'p54.162.7', 'p54.162.8',
       ...
       'v100.16', 'v100.17', 'v100.18', 'v100.19', 'v100.20', 'v100.21',
       'v100.22', 'v100.23', 'v100.24', 'v100.25'],
      dtype='object', length=552)

Furthermore, we set a random seed to make our assigment reproducible.

In [6]:
my_NIA = 100445061
np.random.seed(100445061)

Now, we want to replace some values with: Nan. As "year" will be used to partitionate the data, and "energy" is our response variable, we will leave them aside for this step. Then, we select, at random, 10% of the columns and, again, select 5% of the data inside those columns and replaced them with Nan values. This way, there are 5% of data (from within that 10% column selection) that will take value Nan, but not necesarily 5% in each column.

In [7]:
datos2 = datos.drop(['year','energy'], axis=1)

In [8]:
datos2 = datos.sample(frac=0.10,axis='columns')



In [9]:
nan_mat = np.random.random(datos2.shape)<0.05
nan_mat

array([[False, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [13]:
datos_NaN = datos2.mask(nan_mat)

In [14]:
for col in datos_NaN.columns:
    datos[col]=datos_NaN[col]


After doing this, we save this in a new file called "data.pickle", as this is going to be the dataset we will use.
Please note, that both "year" and "energy" are still in the dataset: as they are fundamental for our research.

In [15]:
datos.to_pickle('data.pickle')

In [16]:
data = pd.read_pickle("data.pickle")

After reading the new file, we divide the dataset in three: training set, validation set, and testing set. We do so, by partitioning through our variable year: 2005-2006, 2007-2008 and 2009-2010. After doing this we drop our variable year, in order to transform the dataframe into a numpy matrix. 

In [17]:
#train
train_data=data.loc[(data.year==2005)|(data.year==2006)]
train_data.drop(['year'],axis='columns', inplace=True)
#validation
validation_data=data.loc[(data.year==2007)|(data.year==2008)]
validation_data.drop(['year'],axis='columns', inplace=True)

#test
test_data=data.loc[(data.year==2009)|(data.year==2010)]
test_data.drop(['year'],axis='columns', inplace=True)


/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Now we transform it to a matrix.

In [18]:
#train
train_y=train_data['energy'].values
train_x=train_data.loc[:,train_data.columns !='energy'].values

#validation
validation_y=validation_data['energy'].values
validation_x=validation_data.loc[:,validation_data.columns !='energy'].values

#test
test_y=test_data['energy'].values
test_x=test_data.loc[:,test_data.columns !='energy'].values


### Part 2

Now, we will first build three preditive models: Knn, Regression Trees and SVMs with default hiper parameters.
For imputation, we will use strategy "median", just to try it. After doing the three models, we will check the "mean absolute error" as our metric for comparison.

In [19]:
imputation=SimpleImputer(strategy='median')
scale=MinMaxScaler()

In [20]:
knn=Pipeline([
    ('imputation',imputation),
    ('scale',scale),
    ('model', KNeighborsRegressor()) ])

In [21]:
svm=Pipeline([
    ('imputation',imputation),
    ('scale',scale),
    ('model', SVR()) ])


In [22]:
tree=Pipeline([
    ('imputation',imputation),
    ('model', DecisionTreeRegressor())
])

In [23]:
knn.fit(train_x,train_y)

Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),
                ('scale', MinMaxScaler()), ('model', KNeighborsRegressor())])

In [24]:
test_y_predict_knn=knn.predict(test_x)

In [25]:
print(mean_absolute_error(test_y,test_y_predict_knn))

365.1681279620853


In [26]:
svm.fit(train_x,train_y)
test_y_predict_svm=svm.predict(test_x)
print(mean_absolute_error(test_y,test_y_predict_svm))

517.5868734870595


In [27]:
tree.fit(train_x,train_y)
test_y_predict_tree=tree.predict(test_x)
print(mean_absolute_error(test_y,test_y_predict_tree))

399.033327014218


In [28]:
scores={'pipeline1':['Knn',365.1681],'pipeline2':['SVM',517.5868],'pipeline3':['Tree',399.03332]}
pd.DataFrame(scores)

,pipeline1,pipeline2,pipeline3
0,Knn,SVM,Tree
1,365.168,517.587,399.033


As we can see, according to the different mean absolute error, our best model is the Knn, followed by the Tree Regression and lastly our Support Vector Machine model.
Then, we will do something similar, but doing hyper parameter tuning.

Before doing this, we follow the instructions and decide to use  PredefinedSplit with no
shuffling for the hyper parameter tuning.

In [29]:
from sklearn.model_selection import PredefinedSplit,RandomizedSearchCV
validation_indices = np.zeros(validation_x.shape[0])
validation_indices[:round(2/3*validation_x.shape[0])] = -1
tr_val_partition = PredefinedSplit(validation_indices)

#### Knn with hyper parameter tuning

For our Knn, we will try to tune (with RandomizedSearchCV) the following 4 parameters:

a) Number of neighbours: this is the number of neighbors picked.

b) Leaf size: This can affect the speed of the construction and query, as well as the memory required to store the tree.

c) Algorithm: this is just the algorithm that will be used to compute the model. It can be: "auto", "ball_tree", "kd_tree", "brute".

d) Weights: this is the function of weight that our model will use. We will either chose uniform weights or the inverse of the distance.

e) Imputing strategy: either using the mean or the median.


In [30]:
knn_params = {
    'model__n_neighbors':np.arange(2,15,1),
    'model__leaf_size':np.arange(25,35,1),
    'model__algorithm':['auto','ball_tree','kd_tree','brute'],
    'model__weights':['uniform','distance'],
    'impute__strategy':['mean','median'],
}
knn = Pipeline([
    ('impute', SimpleImputer()),
    ('scale', scale),
    ('model', KNeighborsRegressor())])
knn_2 = RandomizedSearchCV(estimator=knn,
                            param_distributions=knn_params,
                            n_iter=15,
                            cv=tr_val_partition,
                            scoring='neg_mean_absolute_error')


Now, we construct our Support Vector Machine model, with hyper parameter tuning. We will tune the following parameters:

degree: degree of th epolynomial Kernel function.

gamma: is a Kernel coefficient, and can take values "scale" (1 / (Nº of features * X.var()) or "auto" (1/Nº of features).

shrinking: wether to use the shrinking euristic or not: true or false.

C: this parameter talks about the regularization parameter: and it is inversely proportional to C.

Imputation strategy: either using mean or median.

In [31]:
#svm
svm_params = {
    'model__degree':np.arange(2,15,1),
    'model__gamma':list({'scale','auto'}.union(set(np.logspace(-2, 10, 13)))),
    'model__shrinking':[True, False],
    'model__C':np.logspace(1, 10, 13),
    'impute__strategy':['mean','median']
}

svm = Pipeline([
    ('impute', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('model', SVR())])


svm_2 = RandomizedSearchCV(estimator=svm,
                            param_distributions=svm_params,
                            n_iter=15,
                            cv=tr_val_partition,
                            scoring='neg_mean_absolute_error')

Now, let´s do our Decition Tree Regressor model, with hyper parameter tuning. We will tune the following parameters:

a) criterion:

b) splitter:

c) max_depth:

d) min_samples_split:

e) min_samples_leaf:

f) min_weight_fraction_leaf:

g) max_feature:

h) impuete strategy: either mean or median.


In [32]:
# tree
tree_params = {
    'model__criterion':['mse', 'friedman_mse', 'mae', 'poisson'],
    'model__splitter':['best','random'],
    'model__max_depth':np.arange(2,15,1),
    'model__min_samples_split':np.arange(2,15,1),
    'model__min_samples_leaf':np.arange(1,15,1),
    'model__min_weight_fraction_leaf':np.linspace(0, 0.5, 25),
    'model__max_features':['auto','sqrt','log2'],
    'impute__strategy':['mean','median'],
}

tree = Pipeline([
    ('impute', SimpleImputer()),
    ('model', DecisionTreeRegressor())])

tree_2 = RandomizedSearchCV(estimator=tree,
                            param_distributions=tree_params,
                            n_iter=15,
                            cv=tr_val_partition,
                            scoring='neg_mean_absolute_error')

Now, let´s train and test our new three models.

In [33]:
knn_2.fit(train_x,train_y)

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                             ('scale', MinMaxScaler()),
                                             ('model', KNeighborsRegressor())]),
                   n_iter=15,
                   param_distributions={'impute__strategy': ['mean', 'median'],
                                        'model__algorithm': ['auto',
                                                             'ball_tree',
                                                             'kd_tree',
                                                             'brute'],
                                        'model__leaf_size': array([25, 26, 27, 28, 29, 30, 31, 32, 33, 34]),
                                        'model__n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'model__weights': ['unifor

In [34]:
test_y_predict_knn_2=knn_2.predict(test_x)
print(mean_absolute_error(test_y,test_y_predict_knn_2))

358.5663611316951


In [35]:
svm_2.fit(train_x,train_y)


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                             ('scaler', MinMaxScaler()),
                                             ('model', SVR())]),
                   n_iter=15,
                   param_distributions={'impute__strategy': ['mean', 'median'],
                                        'model__C': array([1.00000000e+01, 5.62341325e+01, 3.16227766e+02, 1.77827941e+03,
       1.00000000e+04, 5.62341325e+04, 3.162...27941e+06,
       1.00000000e+07, 5.62341325e+07, 3.16227766e+08, 1.77827941e+09,
       1.00000000e+10]),
                                        'model__degree': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'model__gamma': [0.1, 1.0, 100000.0,
                                                         1000000.0, 100.0,
                                                    

In [36]:
test_y_predict_svm_2=svm_2.predict(test_x)
print(mean_absolute_error(test_y,test_y_predict_svm_2))

430.90390422116053


In [37]:
tree_2.fit(train_x,train_y)


/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 1242, in fit
    super().fit(
  File "/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 336, in fit
    criterion = CRITERIA_REG[self.criterion](self.n_outputs_,
KeyError: 'poisson'

  warnings.warn("Estimator fit 

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                             ('model',
                                              DecisionTreeRegressor())]),
                   n_iter=15,
                   param_distributions={'impute__strategy': ['mean', 'median'],
                                        'model__criterion': ['mse',
                                                             'friedman_mse',
                                                             'mae', 'poisson'],
                                        'model__max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'mod...
                                        'model__min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'model__min_weight_fraction_leaf': array([0. 

In [38]:
test_y_predict_tree_2=tree_2.predict(test_x)
print(mean_absolute_error(test_y,test_y_predict_tree_2))

344.14989425692903


In [41]:
scores={'pipeline1':['Knn',358.56],'pipeline2':['SVM',430.9],'pipeline3':['Tree',344.15]}
pd.DataFrame(scores)

,pipeline1,pipeline2,pipeline3
0,Knn,SVM,Tree
1,358.56,430.9,344.15


### Part 3

Now, let´s just use our Knn model, and do Feature Selection, Principal Component Analysis. First, only FS, then only PCA and lastly both of them (the three times, in our Knn model).

In [42]:
knn_params_3 = {
    'model__n_neighbors':np.arange(2,15,1),
    'model__leaf_size':[30],
    'model__algorithm':['auto'],
    'model__weights':['uniform'],
    'select__k':list(set(np.arange(2,100,1)).union({'all'})),
    'impute__strategy':['mean']
}
knn=Pipeline([
    ('impute',SimpleImputer()),
    ('scale', scale),
    ('select',SelectKBest()),
    ('model', KNeighborsRegressor())

])
knn_3 = RandomizedSearchCV(estimator=knn,
                            param_distributions=knn_params_3,
                            n_iter=15,
                            cv=tr_val_partition,
                            scoring='neg_mean_absolute_error')

In [43]:
knn_3.fit(train_x,train_y)

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                             ('scale', MinMaxScaler()),
                                             ('select', SelectKBest()),
                                             ('model', KNeighborsRegressor())]),
                   n_iter=15,
                   param_distributions={'impute__strategy': ['mean'],
                                        'model__algorithm': ['auto'],
                                        'model__leaf_size': [30],
                                        'model__n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'model__weights': ['uniform'],
                                        'select__k': [2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                    

In [44]:
test_y_predict_knn_3=knn_3.predict(test_x)
print(mean_absolute_error(test_y,test_y_predict_knn_3))

336.91418088467617


In [45]:

knn_params_4 = {
    'model__n_neighbors':np.arange(2,15,1),
    'model__leaf_size':[30],
    'model__algorithm':['auto'],
    'model__weights':['uniform'],
    'pca__n_components':list(set(np.arange(2,100,1)).union({'all','mle'})),
    'impute__strategy':['mean']

}
knn=Pipeline([
    ('impute',SimpleImputer()),
    ('scale', scale),
    ('pca',PCA()),
    ('model', KNeighborsRegressor()) 
])
knn_4 = RandomizedSearchCV(estimator=knn,
                            param_distributions=knn_params_4,
                            n_iter=15,
                            cv=tr_val_partition,
                            scoring='neg_mean_absolute_error')

In [46]:
knn_4.fit(train_x,train_y)

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                             ('scale', MinMaxScaler()),
                                             ('pca', PCA()),
                                             ('model', KNeighborsRegressor())]),
                   n_iter=15,
                   param_distributions={'impute__strategy': ['mean'],
                                        'model__algorithm': ['auto'],
                                        'model__leaf_size': [30],
                                        'model__n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'model__weights': ['uniform'],
                                        'pca__n_components': [2, 3, 4, 5, 6, 7,
                                                              8, 9, 10, 11, 12,
                                           

In [47]:
test_y_predict_knn_4=knn_4.predict(test_x)
print(mean_absolute_error(test_y,test_y_predict_knn_4))

360.1497227488152


In [48]:
knn=Pipeline([
    ('impute',SimpleImputer()),
    ('scale', scale),
    ("pca", PCA()),
    ('select',SelectKBest()),
    ('model', KNeighborsRegressor()) 
    
])
knn_params_5 = {
    'model__n_neighbors':np.arange(2,15,1),
    'model__leaf_size':np.arange(25,35,1),
    'model__algorithm':['auto'],
    'model__weights':['uniform'],
    'pca__n_components':list(set(np.arange(2,20,1)).union({'all','mle'})),
    'select__k':list(set(np.arange(2,30,1)).union({'all'})),
    'impute__strategy':['mean']

}
knn_5 = RandomizedSearchCV(estimator=knn,
                            param_distributions=knn_params_5,
                            n_iter=15,
                            cv=tr_val_partition,
                            scoring='neg_mean_absolute_error')

In [49]:
knn_5.fit(train_x,train_y)

/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/Users/mauriciomarcos/opt/anaconda3/lib/python3.8/site-

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                             ('scale', MinMaxScaler()),
                                             ('pca', PCA()),
                                             ('select', SelectKBest()),
                                             ('model', KNeighborsRegressor())]),
                   n_iter=15,
                   param_distributions={'impute__strategy': ['mean'],
                                        'model__algorithm': ['auto'],
                                        'model__leaf_size': array([25, 26, 27, 28, 29, 30, 31, 32, 33, 34]),
                                        'model__n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'model__weights': ['uniform'],
                                        'pca__n_components': [2, 3, 4, 5, 6, 7,
        

In [51]:
test_y_predict_knn_5=knn_5.predict(test_x)
print(mean_absolute_error(test_y,test_y_predict_knn_5))

451.99043917851503


In [52]:
scores={'pipeline1':['Fselection',336.91],'pipeline2':['PCA',360.15],'pipeline3':['Both',451.9]}

In [53]:
pd.DataFrame(scores)

,pipeline1,pipeline2,pipeline3
0,Fselection,PCA,Both
1,336.91,360.15,451.9


As we can see in the results above, our best model (the one with the lowet mean absolute error) is our Knn with Feature Selection, followed by our Knn with Principal Component Analysis and lastly our Knn with both.
When comparing with our Knn from the previous section, we can see that we got a better one now when using Feature Selection: we reduce the mean absolute error from 358.6 to 336.91. This means our model will be more powerful, or at least it will commit less errors. As we are using absolute mean error, and not squared mean errors, let´s note that we are not penalizing the model for values that fall very far away from the prediction.

Now, we can check what are the features selected from our dataset in this best model. As we know already, we have 25 locations, so every variable is measured in 25 different locations (marked by the number after the dot in each column name". In other words: "cape.24" is measuring "cape" in location number 24.

In [55]:
a=knn_3.best_estimator_.named_steps['select'].get_support(indices=True)
todas = data.iloc[:,a]
todas.columns


Index(['cape.24', 'cape.25', 'p59.162.1', 'p59.162.2', 'p59.162.3',
       'p59.162.4', 'p59.162.5', 'p59.162.6', 'p59.162.7', 'p59.162.8',
       'p59.162.9', 'p59.162.10', 'p59.162.11', 'p59.162.12', 'p59.162.13',
       'p59.162.14', 'p59.162.15', 'p59.162.16', 'p59.162.17', 'p59.162.18',
       'p59.162.19', 'p59.162.20', 'p59.162.21', 'p59.162.22', 'p59.162.23',
       'u10n.24', 'u10n.25', 'stl3.24', 'stl3.25', 'iews.1', 'iews.2',
       'iews.3', 'iews.4', 'iews.5', 'iews.6', 'iews.7', 'iews.8', 'iews.9',
       'iews.10', 'iews.11', 'iews.12', 'iews.13', 'iews.14', 'iews.15',
       'iews.16', 'iews.17', 'iews.18', 'iews.19', 'iews.20', 'iews.21',
       'iews.22', 'iews.23', 'iews.24', 'iews.25', 'inss.1', 'inss.2',
       'inss.3', 'inss.4', 'inss.5', 'inss.6', 'inss.7', 'inss.8', 'inss.9',
       'inss.10', 'inss.11', 'inss.12', 'inss.13', 'inss.14', 'inss.15',
       'inss.16', 'inss.17', 'inss.18', 'inss.19', 'inss.20', 'inss.21',
       'inss.22', 'inss.23', 'u100.24', 'u

In [56]:
len(todas.columns)

94

We see that our model takes 94 features. But we are interested in two things:

a) how many times do we take into account location number 13 (Sotavento)?

b) What are the locations with higher frequencies: that is, that are taken more into account.

We can easily see that Location Number 13 is evaluated 4 times. While there are others such as, location Number 24, that is counted 5 times as well as location number 25; locations between 10-16 are counted 4 times as well as locations between Number 1 and Number 8. Locations between 17-23 were taken into account only 3 times, as well as location Number 9.

